<a href="https://colab.research.google.com/github/RobAltena/AdventofCode2024/blob/main/Day6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import numpy as np

from typing import Tuple

data = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

data = requests.get('https://raw.githubusercontent.com/RobAltena/AdventofCode2024/refs/heads/main/advent_day6_input.txt').text[:-1]


grid = np.vstack(list((np.fromiter(line, dtype='c') for line in data.split('\n'))))
grid = np.pad(grid, 1, constant_values=' ')
grid_org = grid.copy()

DIRS: Tuple[Tuple[int, int], ...] = ((-1, 0), (0, 1), (1, 0), (0, -1))

In [ ]:
def make_grid(text: str) -> np.ndarray:
    """Build a padded character grid (dtype='<U1')."""
    arr = np.vstack([np.fromiter(line, dtype='<U1') for line in text.splitlines()])
    # pad with spaces so the guard 'falls off' on encountering ' '
    return np.pad(arr, 1, constant_values=' ')

def find_start(g: np.ndarray) -> Tuple[int, int]:
    """Locate the starting caret '^'."""
    (y, x), = np.argwhere(g == '^')
    return int(y), int(x)

def simulate(g: np.ndarray) -> int:
    """Run the walk and return the number of visited cells marked 'X'."""
    y, x = find_start(g)
    dir_idx = 0  # start facing up (matches '^' in the input)

    while True:
        # mark current cell as visited
        g[y, x] = 'X'
        dy, dx = DIRS[dir_idx]
        ny, nx = y + dy, x + dx
        nxt = g[ny, nx]

        if nxt == ' ':
            # fell off the padded map
            break
        if nxt == '#':
            # obstacle: stay in place, keep caret on current, turn right
            g[y, x] = '^'  # original keeps '^' at the current spot on turn
            dir_idx = (dir_idx + 1) % 4
            continue

        # empty floor: move forward and place caret
        y, x = ny, nx
        g[y, x] = '^'

    return int((g == 'X').sum())

In [ ]:
grid = make_grid(data)
result = simulate(grid)
print("Day 6, part 1:", result)

In [ ]:
grid_org = make_grid(data)
start = find_start(grid_org)

possible_blocks = [tuple(coord) for coord in np.argwhere(grid == 'X') if tuple(coord) != start]


In [ ]:
def simulate_2(grid: np.ndarray) -> Tuple[bool, int]:
    """
    Simulate guard movement.
    Returns:
      (loop_detected, visited_count)
    Notes:
      - Mutates `grid` in-place (marks visited with 'X', keeps '^' where the guard stands).
      - Loop is when we revisit the same (y, x) with the same facing direction.
    """
    y, x = find_start(grid)
    dir_idx = 0
    seen = set()  # (y, x, dir_idx)

    while True:
        # Loop check: if we have been here facing the same way, we are trapped
        state = (y, x, dir_idx)
        if state in seen:
            return True, int((grid == 'X').sum())
        seen.add(state)

        # Mark current tile as visited
        grid[y, x] = 'X'

        dy, dx = DIRS[dir_idx]
        ny, nx = y + dy, x + dx
        nxt = grid[ny, nx]

        if nxt == ' ':
            # fell off the padded map -> no loop
            return False, int((grid == 'X').sum())
        if nxt == '#':
            # turn right in place; keep caret notionally here
            dir_idx = (dir_idx + 1) % 4
            grid[y, x] = '^'
            continue

        # move forward
        y, x = ny, nx
        grid[y, x] = '^'

In [ ]:
loop_count = 0
for blixk in possible_blocks:
    grid = make_grid(data)
    grid[blixk] = '#'
    result = simulate_2(grid)
    if result[0]:
      loop_count += 1

print(loop_count)